In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

# Chemin vers le fichier JSONL
file_path = '/home/carolus/Documents/school/green_ia/data/02_openfoodfacts_sample.jsonl'

# Lire le fichier JSONL en utilisant la méthode read_json de pandas avec l'option lines=True
df = pd.read_json(file_path, lines=True)

# Afficher le DataFrame
print(df)


In [ ]:
# main preprocessing

df.rename(columns={'pnns_groups_1': 'groups'}, inplace=True)
df.rename(columns={'ingredients_tags': 'ingredients_temp'}, inplace=True)
df.rename(columns={'product_name': 'name'}, inplace=True)
df.rename(columns={'ecoscore_tags': 'ecoscore_groups'}, inplace=True)
df.rename(columns={'categories_tags': 'categories'}, inplace=True)
df.rename(columns={'ecoscore_score': 'ecoscore_note'}, inplace=True)
df.rename(columns={'labels_tags': 'labels'}, inplace=True)

# traitement col GROUPS 
df['groups'] = df['groups'].replace("unknown", None, regex=False)  


# traitement col INGREDIENTS
df['ingredients_temp'] = df['ingredients_temp'].replace("", None)  # remplace vide par None
df['ingredients_temp'] = df['ingredients_temp'].replace({np.nan: None}) # remplace NaN par None

df['ingredients_temp'] = df['ingredients_temp'].apply(lambda x: x if isinstance(x, list) else []) # remplace None par liste vide 
df['ingredients_temp'] = df['ingredients_temp'].apply(lambda x: ', '.join(x)) # converti liste en string 

# extraire éléments avec 'en:' nouvelle colonne
def extract_en_ingredients(ingredient_list):
    ingredients = ingredient_list.strip('[]').split(', ')
    return [ingredient.split(':')[-1] for ingredient in ingredients if ingredient.startswith('en:')]
df['ingredients'] = df['ingredients_temp'].apply(extract_en_ingredients)
df.drop(columns=['ingredients_temp'], inplace=True)
df['ingredients'] = df['ingredients'].apply(lambda x: ', '.join(x))
df['ingredients'] = df['ingredients'].replace("", None)  

In [ ]:
df.tail(50)

In [ ]:
df['ingredients'].head(10)

In [ ]:
nan_counts = df.isna().sum()

# Compter les listes vides
empty_list_counts = df.map(lambda x: x == []).sum()

# Afficher les résultats
print("nombre de NaN ou None par colonne :")
print(nan_counts)

In [ ]:
print("nombre de listes vides par colonne :")
print(empty_list_counts)

In [ ]:
df['labels'].head(50)

In [ ]:
df['ingredients'].head(50)